In [1]:
from arcgis.gis import GIS
gis=GIS("home")

In [4]:
import arcpy
import requests
from zipfile import ZipFile
import io
import os
import pandas as pd

In [5]:
##insert date as "YYYY-MM-DD" to get data for selected date
date = "2024-11-11"
ndawn_url= f"https://ndawn.ndsu.nodak.edu/table.csv?station>0&variable=ddmxt&variable=ddmxtt&variable=ddmnt&variable=ddmntt&variable=ddavt&year=2024&ttype=daily&quick_pick=30_d&begin_date={date}&end_date={date}"
ndawn_res = requests.get(ndawn_url, allow_redirects=True)

## turn CSV into dataframe, ignoring the text above the header row, and the units row
ndawn_csv_to_df = pd.read_csv(io.StringIO(ndawn_res.content.decode("utf-8")), header = 3, skiprows=[4])

ndawn_csv_to_df


,Station Name,Latitude,Longitude,Elevation,Year,Month,Day,Max Temp,Max Temp Flag,Max Temp Time,Max Temp Time Flag,Min Temp,Min Temp Flag,Min Temp Time,Min Temp Time Flag,Avg Temp,Avg Temp Flag
0,Ada,47.32119,-96.51406,910,2024,10,27,60.224,NaN,15:17:00,NaN,32.099,NaN,05:40:00,NaN,46.162,NaN
1,Ada,47.32119,-96.51406,910,2024,10,28,65.480,NaN,14:12:00,NaN,44.130,NaN,06:59:00,NaN,54.805,NaN
2,Ada,47.32119,-96.51406,910,2024,10,29,58.658,NaN,13:05:00,NaN,43.349,NaN,23:59:00,NaN,51.004,NaN
3,Ada,47.32119,-96.51406,910,2024,10,30,46.328,NaN,14:14:00,NaN,28.666,NaN,00:00:00,NaN,37.497,NaN
4,Ada,47.32119,-96.51406,910,2024,10,31,41.322,NaN,15:51:00,NaN,23.778,NaN,06:43:00,NaN,32.550,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6529,Zeeland,46.01351,-99.68768,2070,2024,11,21,23.389,NaN,14:32:00,NaN,7.286,NaN,23:51:00,NaN,15.338,NaN
6530,Zeeland,46.01351,-99.68768,2070,2024,11,22,37.450,NaN,14:20:00,NaN,7.988,NaN,00:01:00,NaN,22.719,NaN
6531,Zeeland,46.01351,-99.68768,2070,2024,11,23,27.529,NaN,14:32:00,NaN,21.335,NaN,00:00:00,NaN,24.432,NaN
6532,Zeeland,46.01351,-99.68768,2070,2024,11,24,26.530,NaN,14:15:00,NaN,11.894,NaN,00:00:00,NaN,19.212,NaN


In [ ]:
##Run an IDW analysis
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\rockx057\Documents\ArcGIS\Projects\lab 3 part 2\Default.gdb"):
    out_raster = arcpy.sa.Idw(
        in_point_features="station_points",
        z_field="max_temp",
        cell_size=0.021010908,
        power=2,
        search_radius="VARIABLE 12",
        in_barrier_polyline_features=None
    )
    out_raster.save(r"C:\Users\rockx057\Documents\ArcGIS\Projects\lab 3 part 2\Default.gdb\Idw_station_1")

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"C:\Users\rockx057\Documents\ArcGIS\Projects\lab 3 part 2\Default.gdb"):
    out_surface_raster = arcpy.sa.Kriging(
        in_point_features="station_points",
        z_field="max_temp",
        kriging_model="Spherical # # # #",
        cell_size=0.021010908,
        search_radius="VARIABLE 12",
        out_variance_prediction_raster=None
    )
    out_surface_raster.save(r"C:\Users\rockx057\Documents\ArcGIS\Projects\lab 3 part 2\Default.gdb\Kriging_stat2")

In [ ]:
arcpy.ga.LocalPolynomialInterpolation(
    in_features="station_points",
    z_field="max_temp",
    out_ga_layer="geostats_lpi_1",
    out_raster=r"C:\Users\rockx057\Documents\ArcGIS\Projects\lab 3 part 2\Default.gdb\LPI_raster_1",
    cell_size=0.021010908,
    power=1,
    search_neighborhood="NBRTYPE=Standard S_MAJOR=3.42563109432527 S_MINOR=3.42563109432527 ANGLE=0 NBR_MAX=15 NBR_MIN=10 SECTOR_TYPE=ONE_SECTOR",
    kernel_function="EXPONENTIAL",
    bandwidth=None,
    use_condition_number="NO_USE_CONDITION_NUMBER",
    condition_number=None,
    weight_field=None,
    output_type="PREDICTION"
)